This notebook consist of yolov5 on custom data with difference inferences, 

#Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
!git clone https://github.com/MNaseerSubhani/EasyYolov5Pytorch.git

In [ ]:
%cd EasyYolov5Pytorch

In [ ]:
!git submodule init
!git submodule update

In [ ]:
%cd yolov5

In [ ]:
!pip install -r requirements.txt  # install

In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images
clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

###  <font color='yellow'>Yolo Instance Init</font> : <font color='red'>RunFirstTime</font>  when creating new Instance

In [59]:
%cd .. 

/content/drive/My Drive/EasyYolov5Pytorch


In [60]:
instance_name = "sugar_bag"     # Put any name of the Instance 
#set hyperparameters
num_of_classes = 1                  # Total number of classes       
size = 416                         # size of input image
batch = 32                          # batch size
model = 'yolov5s'                          # "yolov5s" or "yolov5m" or "yolov5x" or "yolov5l"

In [61]:
import os
if not os.path.exists(instance_name):
  os.mkdir(instance_name)

#Create dataset.yaml

In [ ]:
%cd {instance_name}
with open("dataset.yaml", "w") as f:   
    f.write(f"""path: ../{instance_name}/data  # dataset root dir
train: train/images  # train images (relative to 'path') 128 images
val: test/images  # val images (relative to 'path') 128 images
test:  # test images (optional)\n\n
nc: {num_of_classes}  # number of classes\n\n
names: [ 'bag' ]""")
%cd ..

###  <font color='red'>NOTE: At this stage add dataset to the instance folder as structured below</font>

--data

  ----------- train

  -------------- images
  
  -------------- labels

  ----------- test

###Copy and Edit model .yaml file : <font color='red'>RunFirstTime</font>  when creating new Instance

In [63]:
from shutil import copyfile
copyfile(f'custom_models/{model}.yaml', f'{instance_name}/{model}.yaml')

'sugar_bag/yolov5s.yaml'

In [64]:
%cd {instance_name}
f_nm = f'{model}.yaml'
!sed -i 's/nc: 3/nc: $num_of_classes/' {f_nm}
%cd ..

/content/drive/My Drive/EasyYolov5Pytorch/sugar_bag
/content/drive/My Drive/EasyYolov5Pytorch


#Train

In [65]:
%cd yolov5

/content/drive/My Drive/EasyYolov5Pytorch/yolov5


###download pretrained model 

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/{model}.pt


In [68]:
%pip install -q wandb
import wandb
wandb.login()

False

In [ ]:
# Train YOLOv5s on COCO128 for 5 epochs
!python3 train.py --img 320 --batch {batch} --epochs 200 --data ../{instance_name}/dataset.yaml  --weights {model}.pt



In [ ]:
import shutil
shutil.move("runs/train/exp/weights/best.pt", f"../{instance_name}/best.pt")

#Inference

In [ ]:
fn = 'test.jpg'
!python3 detect.py --weights ../{instance_name}/best.pt --img 640 --conf 0.25 --source ../{instance_name}/data/test/images/{fn}
Image(filename=f'runs/detect/exp5/{fn}', width=600)